# Visualizing molecules with virtual sites via Interchange

This example appends virtual site parameters to Sage, parametrizes some small molecules with it, and visualizes the result.

In [ ]:
import tempfile

import nglview
from openff.toolkit import ForceField, Molecule
from openff.toolkit.typing.engines.smirnoff.parameters import (
    VirtualSiteHandler,
    VirtualSiteType,
)
from openff.units import unit

from openff.interchange import Interchange

There are several ways to create `ForceField`s or add parameters to existing objects. Here, we will use the OpenFF Toolkit API to create two virtual site parameters and then add them to OpenFF 2.1.0 code name "Sage."

We will create two parameters: one representing a nitrogen lone pair in an aromatic ring and another to fill in the sigma hole in bromine bonded to aliphatic carbon.

In [ ]:
nitrogen_lone_pair = VirtualSiteType(
    smirks="[#6X3H1a:2]1:[#7X2a:1]:[#6X3H1a:3]:[#6X3a]:[#6X3a]:[#6X3a]1",
    type="DivalentLonePair",
    match="once",
    name="EP1",
    distance=0.5 * unit.angstrom,
    outOfPlaneAngle=0.0 * unit.degree,
    epsilon=0.0 * unit.kilojoule_per_mole,
    sigma=0.0 * unit.nanometer,
    charge_increment=[0.2, 0.0, 0.0] * unit.elementary_charge,
)

bromine_sigma_hole = VirtualSiteType(
    smirks="[#6A:2]-[#35:1]",
    type="BondCharge",
    match="all_permutations",
    name="EP2",
    distance=-1.5 * unit.angstrom,
    epsilon=0.0 * unit.kilojoule_per_mole,
    sigma=0.0 * unit.nanometer,
    charge_increment=[0.1, 0.0] * unit.elementary_charge,
)

Now that we've created our two parameters as `VirtualSiteType` objects, we can create a handler (`VirtualSiteHandler`), add the two parameters to the handler, and finally add the handler to the force field.

In [ ]:
virtual_sites = VirtualSiteHandler(version=0.3)

virtual_sites.add_parameter(parameter=nitrogen_lone_pair)
virtual_sites.add_parameter(parameter=bromine_sigma_hole)

force_field = ForceField("openff-2.1.0.offxml")
force_field.register_parameter_handler(virtual_sites)

print(f"Registered handlers: {force_field.registered_parameter_handlers}\n")
print("Virtual site parameters:")
for parameter in force_field["VirtualSites"].parameters:
    print(parameter)

Now that our force field includes virtual site parameters, we can use it to parametrize molecules!

Interchange provides a `.to_pdb` method that optionally includes virtual sites, computing their positions from the geometry specified in the force field. To quickly look at some single-molecule results, let's wrap this up into a function that takes a SMILES pattern, writes out a PDB file via Interchange, and visualizes the result with [NGLview](https://nglviewer.org/#nglview).

In [ ]:
def visualize(smiles: str) -> nglview.NGLWidget:
    molecule = Molecule.from_smiles(smiles)
    molecule.generate_conformers(n_conformers=1)

    interchange = force_field.create_interchange(molecule.to_topology())

    with tempfile.NamedTemporaryFile(suffix=".pdb") as temporary_file:
        interchange.to_pdb(temporary_file.name, include_virtual_sites=True)

        view = nglview.show_file(temporary_file.name)

    view.clear_representations()
    view.add_representation("hyperball")

    return view

In [ ]:
visualize("c1ncccc1")

In [ ]:
visualize("BrCCCCBr")

Think about how these parameters could be modified to produce different physics.

* How would you change the nitrogen lone pair virtual site to
  * Be placed closer to the nitrogen atom/nucleus?
  * Not sit flat in the plane of the ring?
  * Move a charge of 0.3 e instead of 0.2?
* How would you change the bromine parameter to
  * Apply to chlorocarbons instead?
  * Have (weak) vdW interactions?
  * Move a charge of 0.2 e instead of 0.1?
